# Extending DLT Pipelines with New Notebooks

In [0]:
SET school.dataset_path=dbfs:/mnt/DE-Associate-Book/datasets/school;

In [0]:
CREATE OR REFRESH STREAMING TABLE courses_bronze
COMMENT "The raw courses data, ingested from CDC feed"
AS SELECT * FROM cloud_files("${school.dataset_path}/courses-cdc", "json")

In [0]:
CREATE OR REFRESH STREAMING TABLE courses_silver;

In [0]:
APPLY CHANGES INTO LIVE.courses_silver
 FROM STREAM(LIVE.courses_bronze)
 KEYS (course_id)
 APPLY AS DELETE WHEN row_status = "DELETE"
 SEQUENCE BY row_time
 COLUMNS * EXCEPT (row_status, row_time)

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW instructor_counts_stats
 COMMENT "Number of courses per instructor"
AS SELECT instructor, count(*) as courses_count,
         current_timestamp() updated_time
 FROM LIVE.courses_silver
 GROUP BY instructor

In [0]:
CREATE TEMPORARY LIVE VIEW courses_sales
 AS SELECT b.title, o.quantity
   FROM (
     SELECT *, explode(courses) AS course
     FROM LIVE.enrollments_cleaned) o
   INNER JOIN LIVE.courses_silver b
   ON o.course.course_id = b.course_id;